# Full 1-loop amplitude calculation for muon production through the process:

$$ \huge e^+ e^- \rightarrow \mu^+ \mu^- $$

Scattering amplitude calculations can be divided into three steps:

* **Generation**: Writing the amplitude as a sum of Feynman diagrams.
* **Reduction**: Expressing the loop integrals as a linear combination of a set of master integrals.
* **Computation**: Evaluating the master integrals.

This notebook demonstrates how the computation step is done in *pySecDec*, for the process of $e^+ e^- \rightarrow \mu^+ \mu^-$ at 1-loop. The generation of Feynman diagrams, the interference with the Born amplitude as well as the Passarino-Veltman (PaVe) reduction to scalar master integrals, was done with *FeynCalc* [1,2,3]. 

#### Import the required pySecDec modules for the sector decomposition

In [ ]:
import pySecDec as psd
from pySecDec import LoopPackage
from pySecDec import sum_package

### Define the coefficients of the scalar master integrals

The coefficients of the scalar PaVe integrals are copied directly from the FeynCalc output into a list structure. Element *j* in this list is the coefficient of the scalar integral stored as element *j* in another list of the scalar integrals defined later. 

For this process at 1-loop there is a diagram containing a closed lepton loop:

<img src="fermionloop.jpeg" 
     align="center"
     width="250" />

This means there will be some coefficients proportional to the number of leptons $N_f$. These coefficients are placed into another list so that they can be integrated separately. The amplitude is in this way split up into two smaller building blocks that are gauge invariant on their own. The final result will thus have a part proportional to $N_f$, which can be considered separately if desired. 

Finally, an integral prefactor which relates FeynCalc's and pySecDec's loop integral definitions is defined. A common prefactor of $2\pi \cdot e^6$ has been removed from the coefficients and will be added back to the results after integration.

In [ ]:
integral_coeffs = ['8*((4 - 2*eps)^2*s^2 - 3*(4 - 2*eps)*t^2 - 10*(4 - 2*eps)*t*u - 3*(4 - 2*eps)*u^2 + 2*t^2 + 8*t*u + 2*u^2)/((3 - 2*eps)*s^2)',
    '2*((4 - 2*eps)^3*s^2 - 11*(4 - 2*eps)^2*t^2 + 26*(4 - 2*eps)*t^2 - 24*(4 - 2*eps)^2*t*u + 78*(4 - 2*eps)*t*u - 9*(4 - 2*eps)^2*u^2 + 20*(4 - 2*eps)*u^2 - 16*t^2 - 56*t*u - 12*u^2)/((3 - 2*eps)*s^2)',
    '((4 - 2*eps)*t + 3*(4 - 2*eps)*u - 4*t - 8*u)/s',
    '-(3*(4 - 2*eps)*t + (4 - 2*eps)*u - 8*t - 4*u)/s',
    '-2*((4 - 2*eps)^2*s^2 - 5*(4 - 2*eps)*s^2 + 4*(4 - 2*eps)*u^2 + 4*t^2 + 8*t*u)/((3 - 2*eps)*s)',
    '-t*((4 - 2*eps)^2*s + 9*(4 - 2*eps)*t + (4 - 2*eps)*u - 8*t)/(2*(3 - 2*eps)*s)',
    'u*((4 - 2*eps)^2*s + (4 - 2*eps)*t + 9*(4 - 2*eps)*u - 8*u)/(2*(3 - 2*eps)*s)',
    '-2*((4 - 2*eps)^2*s^2 - 5*(4 - 2*eps)*s^2 + 4*(4 - 2*eps)*u^2 + 4*t^2 + 8*t*u)/((3 - 2*eps)*s)',
    '-t*((4 - 2*eps)^2*s + 9*(4 - 2*eps)*t + (4 - 2*eps)*u - 8*t)/(2*(3 - 2*eps)*s)',
    'u*((4 - 2*eps)^2*s + (4 - 2*eps)*t + 9*(4 - 2*eps)*u - 8*u)/(2*(3 - 2*eps)*s)',
    '-t*(3*(4 - 2*eps)^2*s^2 - 3*(4 - 2*eps)*t^2 - 30*(4 - 2*eps)*t*u - 11*(4 - 2*eps)*u^2 + 24*t*u + 8*u^2)/(2*(3 - 2*eps)*s)',
    'u*(3*(4 - 2*eps)^2*s^2 - 11*(4 - 2*eps)*t^2 - 30*(4 - 2*eps)*t*u - 3*(4 - 2*eps)*u^2 + 8*t^2 + 24*t*u)/(2*(3 - 2*eps)*s)']

integral_coeffs_N = ['0', '2*(-(4 - 2*eps)^2*s^2 + 4*(4 - 2*eps)*t^2 + 12*(4 - 2*eps)*t*u + 4*(4 - 2*eps)*u^2 - 4*t^2 - 16*t*u - 4*u^2)/((3 - 2*eps)*s^2)', '0', '0', '0', '0', '0', '0', '0', '0' ,'0' ,'0']

additional_prefactor = 'gamma(1-2*eps)/(gamma(1-eps)*gamma(1-eps)*gamma(1 + eps))'

#### Format the Mathematica expressions
Since *FeynCalc* produces Mathematica expressions, the algebraic expressions need to be slightly modified to fit the Python syntax.

In [ ]:
coeffs = []
for integral_coeff in integral_coeffs:
    formatted_coeff = integral_coeff.replace('^', '**')
    formatted_coeff = formatted_coeff.replace(' ', '')
    coeffs.append(formatted_coeff)
    
N_coeffs = []
for integral_coeff in integral_coeffs_N:
    formatted_coeff = integral_coeff.replace('^', '**')
    formatted_coeff = formatted_coeff.replace(' ', '')
    N_coeffs.append(formatted_coeff)

coefficients = [coeffs, N_coeffs]

### Defining scalar PaVe integrals within the *pySecDec* framework

Functions to generate the massless scalar PaVe integrals $B_0(p^2)$, $C_0(p_1^2, p_2^2, p_{12}^2)$ and $D_0(p_1^2, p_2^2, p_3^2, p_4^2, p_{12}^2, p_{23}^2)$ as *pySecDec LoopPackages* are defined. The corresponding Feynman diagrams are used to create *pySecDec loop integrals* with the function *LoopIntegralFromGraph*. The functions return instances of *LoopPackage*, which prepares the *loop integrals* for sector decomposition. 

*LoopPackage* requires a few parameters. The *real parameters* are the kinematics of the process, in this case only squares of external momenta as masses are not taken into account, *decomposition_method* is the type of sector decomposition and *requested_orders* is the highest order in the expansion of the regularization parameter. The *additional_prefactor* attaches the common prefactor that was defined earlier to each *loop integral*. 

In [ ]:
def B0(p_sq, name):
        li = psd.LoopIntegralFromGraph(
                internal_lines = [[0,[1,2]],[0,[2,1]]],
                external_lines = [['p',1],['p',2]],
                replacement_rules = [('p*p', p_sq)])
        real_parameters = []
        if not p_sq == 0: #Pass the momentum as a symbolic parameter if it is not 0
            real_parameters.append(p_sq)
        return LoopPackage(name, loop_integral = li, real_parameters = real_parameters, 
                            decomposition_method = 'geometric', requested_orders = [0], additional_prefactor = additional_prefactor)

def C0(p1_sq, p2_sq, p12_sq, name):
    li = psd.LoopIntegralFromGraph(
            internal_lines = [[0,[1,2]],[0,[2,3]],[0,[3,1]]],
            external_lines = [['p1',1],['p2',2],['p3',3]],
            replacement_rules = [
                                ('p1*p1', p1_sq),
                                ('p2*p2', p2_sq),
                                ('p3*p3', p12_sq)
                                ])
    real_parameters = [p for p in [p1_sq, p2_sq, p12_sq] if p != 0] #Pass the momenta as symbolic parameters if they are not 0
    return LoopPackage(name, loop_integral = li, real_parameters = real_parameters, 
                        decomposition_method = 'geometric', requested_orders = [0], additional_prefactor = additional_prefactor)

def D0(p1_sq, p2_sq, p3_sq, p4_sq, p12_sq, p23_sq, name):
    li = psd.LoopIntegralFromGraph(
            internal_lines = [[0,[1,2]],[0,[2,3]],[0,[3,4]],[0,[4,1]]],
            external_lines = [['p1',1],['p2',2],['p3',3], ['p4',4]],
            replacement_rules = [
                                ('p1*p1', p1_sq),
                                ('p2*p2', p2_sq),
                                ('p3*p3', p3_sq),
                                ('p4*p4', p4_sq),
                                ('p3*p2', str(p23_sq) + '/2' + '-' + str(p2_sq) + '/2' + '-' + str(p3_sq) + '/2'),
                                ('p1*p2', str(p12_sq) + '/2' + '-' + str(p1_sq) + '/2' + '-' + str(p2_sq) + '/2'),
                                ('p1*p4', str(p23_sq) + '/2' + '-' + str(p1_sq) + '/2' + '-' + str(p4_sq) + '/2'),
                                ('p2*p4', '-' + str(p12_sq) + '/2' + '-' + str(p23_sq) + '/2' + '-' + str(p2_sq) + '/2' + '-' + str(p4_sq) + '/2'),
                                ('p1*p3', '-' + str(p12_sq) + '/2' + '-' + str(p23_sq) + '/2' + '-' + str(p1_sq) + '/2' + '-' + str(p3_sq) + '/2'),
                                ('p3*p4', str(p12_sq) + '/2' + '-' + str(p3_sq) + '/2' + '-' + str(p4_sq) + '/2')
                                ])
    real_parameters = [p for p in [p1_sq, p2_sq, p3_sq, p4_sq, p12_sq, p23_sq] if p != 0] #Pass the momenta as symbolic parameters if they are not 0                                     
    return LoopPackage(name, loop_integral = li, real_parameters = real_parameters, 
                        decomposition_method = 'geometric', requested_orders = [0], additional_prefactor = additional_prefactor)

#### Instantiating the LoopPackages
The functions defined above are used to define the scalar integrals as *pySecDec LoopPackages*. They are ordered in a list in the same way as the coefficients were before.  

In [ ]:
B0_integrals = [B0(0, 'B00'), B0('s', 'B0s'), B0('t', 'B0t'), B0('u', 'B0u')]
C0_integrals = [C0(0, 0, 's', 'C00s'), C0(0, 0, 't', 'C00t'), C0(0, 0, 'u', 'C00u'), C0(0, 's', 0, 'C0s0'), C0(0, 't', 0, 'C0t0'), C0(0, 'u', 0, 'C0u0')]
D0_integrals = [D0(0,0,0,0, 's', 't', 'D0000st'), D0(0,0,0,0, 's', 'u', 'D0000su')]

all_integrals = B0_integrals + C0_integrals + D0_integrals #integral[0] corresponds to coeffs[0] etc.

### Bringing everything together and generating the C++ integration code

The function *sum_package* combines the scalar integrals with the corresponding coefficients and builds the C++ integration code, i.e. the amplitude is constructed. The real parameters are the kinematics of the process that are specified before integration. In this case they are the standard Mandelstam invariants as there are no masses taken into account.


In [ ]:
sum_package(
        'muon_production',
        all_integrals,
        coefficients = coefficients,
        requested_orders = [0],
        regulators = ['eps'],
        real_parameters = ['s', 't', 'u']) 
#Make sure that the list of real parameters contain every symbolic kinematic invariant defined in 'all_integrals'

#### Build the C++ library
Compile the C++ code and prepare the loop integrals for integration. This make routine will build the Quasi Monte Carlo (QMC) integrator *Deval*. 

In [ ]:
%cd muon_production
import os
os.system('make disteval.done')
%cd ..

#### Import the pySecDec integral library, as well as sympy and numpy which are used to format the results

In [ ]:
from pySecDec.integral_interface import DistevalLibrary
import sympy as sp
import numpy as np

#### Load the built integral library

In [ ]:
amplitude = DistevalLibrary('muon_production/disteval/muon_production.json')

### Perform the integration and retrieve the result

The integration is performed for an arbitrary phase space point ``s = 3.0, t = -1.0, u = -2.0``. As mentioned before the results are split due to some coefficients depending on the number of leptons.

In [ ]:
str_result = amplitude(parameters={"s": 3.0, "t": -1.0, "u": -2.0}, verbose=True)

result = sp.sympify(str_result)
value = result[0].subs({"plusminus": 0})
valueN =result[1].subs({"plusminus": 0})
error = result[0].coeff("plusminus")
errorN = result[1].coeff("plusminus")

#### Formating results
Express results in terms of the QED fine structure constant $\alpha$ and reinclude the factor of $2\pi$ which had been removed from the coefficients earlier. In natural units the relation between the QED fine structure constant and the electric charge is $e^2=4\pi\alpha$. 

In [ ]:
value *= np.pi**5 * 4**3
error *= np.pi**5 * 4**3 
valueN *= np.pi**5 * 4**3
errorN *= np.pi**5 * 4**3

#### Printing results
Results are printed at three different orders of the regularization parameter. 

In [ ]:
print('Numerical Result Proportional to alpha**3 (Nf is the number of leptons):')
print('eps^-2:', value.coeff('eps',-2), '+/- (', error.coeff('eps',-2), ')', '\n', '        + Nf*(', valueN.coeff('eps',-2), '+/- (', errorN.coeff('eps',-2), '))', '\n')
print('eps^-1:', value.coeff('eps',-1), '+/- (', error.coeff('eps',-1), ')', '\n', '        + Nf*(', valueN.coeff('eps',-1), '+/- (', errorN.coeff('eps',-1), '))', '\n')
print('eps^0:', value.coeff('eps',0), '+/- (', error.coeff('eps',0), ')', '\n', '       + Nf*(',valueN.coeff('eps',0), '+/- (', errorN.coeff('eps',0), '))')

#### Plotting results
By integrating for different phase space points, and maintaining the physical condition that $s+u+t = 0$, we may plot a line in phase space. Since no renormalisation is done, we are restricted to plot the coefficients of the different powers of $\varepsilon$. Here we settle for the real and imaginary parts of the coefficient to $\varepsilon^0$. To only get numbers, $N_f = 3$ has been chosen.

In [ ]:
from matplotlib import pyplot as plt
real_results = []
complex_results = []
kinematics = range(2,21)
for s in kinematics: #let s be all integers from 2 to 20. 
    t = -1.0
    u = -s-t #Preserve the physical condition for the kinematics
    
    amplitude = DistevalLibrary('muon_production/disteval/muon_production.json') # load the library
    str_result = amplitude(parameters={"s": s, "t": t, "u": u}, verbose=True) # integrate
    
    #Extract results
    result = sp.sympify(str_result)
    value = result[0].subs({"plusminus": 0})
    valueN =result[1].subs({"plusminus": 0})
    error = result[0].coeff("plusminus")
    errorN = result[1].coeff("plusminus")

    #Express results in terms of fine structure constant (and add back common pi factors of coefficients)
    value *= np.pi**5 * 4**3
    error *= np.pi**5 * 4**3 
    valueN *= np.pi**5 * 4**3
    errorN *= np.pi**5 * 4**3

    real_result = sp.re(value.coeff('eps',0)) + 3 * sp.re(valueN.coeff('eps',0))
    complex_result = sp.im(value.coeff('eps',0)) + 3 * sp.im(valueN.coeff('eps',0))
    real_results.append(real_result)
    complex_results.append(complex_result)
    
plt.figure()
plt.title('Line in phase space; s+t+u = 0')
plt.xlabel('Kinematic invariant: s')
plt.plot(kinematics, complex_results, 'ro', label = 'Im(coeff to eps^0)')
plt.plot(kinematics, real_results, 'bo', label = 'Re(coeff to eps^0)')
plt.legend(loc='upper left')
plt.grid(True)
plt.savefig("muon_production_plot.pdf", format='pdf', bbox_inches='tight')


### References
[1] V. Shtabovenko, R. Mertig and F. Orellana, "*FeynCalc 9.3: New features and improvements*", arXiv:2001.04407   
[2] V. Shtabovenko, R. Mertig and F. Orellana, "*New Developments in FeynCalc 9.0"*, Comput. Phys. Commun., 207, 432-444, 2016, arXiv:1601.01167  
[3] R. Mertig, M. Böhm, and A. Denner, *"Feyn Calc - Computer-algebraic calculation of Feynman amplitudes"*, Comput. Phys. Commun., 64, 345-359, 1991